# Estimating stellar and planetary parameters based on visual inspection  

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

<a href="https://colab.research.google.com/github/rges-pit/data-challenge-notebooks/blob/main/Extras/Gould_Loeb_Planetary_Event.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you would like an introduction to python notebooks, please read this tutorial: https://medium.com/codingthesmartway-com-blog/getting-started-with-jupyter-notebook-for-python-4e7082bd5d46

Please note, you must **save this notebook in a space owned by you** (a GitHub repo, gist, to Google Drive, or locally) if you want to come back to it later without losing your progress. You can edit and run this notebook on Colab, but it **will not auto save** for you.

### <font face="Helvetica" size="5"> Running locally and local runtimes </font>

*Ignore this section if you already did these steps for another notebook.*

If you choose to use lcoal resources your notebook will use your local packages, so you should install a virtual environment with the following packages. Run the cell below to create a downloadable `.yml` file, to automate the package install process (provided you are using anaconda). Failing that, you can download it from the [git repository](https://github.com/rges-pit/minicourses.git).

In [ ]:
# @title `.yml`
yaml = '''name: minicourse
channels:
  - conda-forge
dependencies:
  - python=3.11
  - numpy
  - matplotlib
  - pandas
  - scipy
  - jupyter
  - ipython
  - astropy
  - beautifulsoup4
  - lxml              # required parser for bs4
  - pip
  - pip:
      - pathos
      - MulensModel
      - pyLIMA'''

# save the yaml
with open('environment.yml', 'w') as f:
    f.write(yaml)

Click the folder button on the side bar to open the file explorer. The file `environment.yml` should be in there now. Just click the triple dots on the side and then `Download` to download the `.yml` file.

```bash
conda env create -f environment.yml
```

Running the above line in a terminal (Anaconda Prompt on Windows) will create a virtual conda environment called `minicourse`, which has the required packages installed.

You can activate the environment with:

```bash
conda activate minicourse
```

From here you have two options

1. You can open the notebook running
```bash
jupyter notebook
```
from a parent folder to your locally saved version of this notebook and navigating to the notebook in your browser. You may need to select `minicourse` as your kernel before running the notebook.

2. Alternatly, you can create a local "Runtime" and for your Colab notebook by following [these instructions](https://www.google.com/url?q=https%3A%2F%2Fresearch.google.com%2Fcolaboratory%2Flocal-runtimes.html).
```bash
jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --no-browser
```

  ⚠️ We don't generally recommend that you do this with notebooks that you didn't write as it give them access to your local machine

<!-- -->

### <font face="Helvetica" size="5"> Clone the rges-pit/minicourses repo </font>

Do this weather you are running locally or on the cloud, if you do not already have the repo cloned. It has necessary data.

In [ ]:
# #@title Clone the rges-pit/minicourses repo
!git clone https://github.com/rges-pit/minicourses.git

In [ ]:
!ls -a ./minicourses/
%cd minicourses/
!git status

In [ ]:
!git pull
!git status

## <font face="Helvetica" size="6"> 1. Introduction </font>

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

Weolcome to **Chapter 4: Estimating stellar and planetary parameters based on visual inspection**




This exercise was adapted into a Jupyter Notebook from the exercise from Sagan Summer Exoplanet Workshop 2017, made by Jennifer Yee. See Tuesday's hands-on materials and accompanying worksheet available here: [**link**](https://nexsci.caltech.edu/workshop/2017/handson.shtml)

We will analyze one of the simplest cases of a microlensing event with a planetary anomaly, which was described in [**Gould & Loeb, 1992**](https://ui.adsabs.harvard.edu/abs/1992ApJ...396..104G/abstract).

We make a couple of assumptions here. We are trying to analyze a Gould & Loeb planetary event with a caustic crossing perturbation. In such an event, we assume that there is **no parallax** and **no blending**.

## <font face="Helvetica" size="6"> 2. Set-up </font>

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

### <font face="Helvetica" size="5"> 2.1 Load the packages </font>

We will start be getting out runtime/kernel set up with all the necessary packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# matplotlib widget unfortunately doesn't work with Google Collab :(
# %matplotlib widget

### <font face="Helvetica" size="5"> 2.2 Load the data and display the event </font>

In [ ]:
#Load data (row 1 is header)
df = pd.read_csv('/home/meet/data-challenge-notebooks/AAS Workshop/Session C: Binary Lens/2012-BLG-0406_data.txt', sep='\t', header=0)
#load parameters into a dictionary
params = np.loadtxt('/home/meet/data-challenge-notebooks/AAS Workshop/Session C: Binary Lens/2012-BLG-0406_params.txt', unpack=True, skiprows=1, dtype=str)
params_dict = dict(zip(params[0], np.float64(params[1])))
#select only data points where t is in range t0 - 5*tE and t0 + 5*tE
t0i = params_dict['Tmax'] - 2450000
tEi = params_dict['tau']
u0i = params_dict['umin']

df = df[(df['HJD-2450000'] > t0i - 4*tEi) & (df['HJD-2450000'] < t0i + 4*tEi)]

Go ahead and zoom in at the features of the event.

In [ ]:
#@title Plot the event

fig = plt.figure()
plt.errorbar(df['HJD-2450000'], df['I_Mag'], yerr=df['Err'], marker='o', color='black', ls='', markersize=1);
plt.gca().invert_yaxis();
plt.xlabel('HJD-2450000');
plt.ylabel('I-Mag');


## <font face="Helvetica" size="6"> 3. Parameters that we will estimate </font>

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

Let's have a quick reminder about the main parameters that characterize a microlensing event. We will use this plot from [**Khakpash et al. 2019**](https://ui.adsabs.harvard.edu/abs/2019AJ....158....9K/abstract):

<center><img src="https://github.com/rges-pit/minicourses/blob/main/chapter4/images/Khakpash_2019_parameters.png?raw=true" width="500"/></center>
<center>Plot from Khakpash et al. 2019, Figure 1.</center>

This plot's y-axis is in **magnification**, instead of magnitudes or fluxes. Then the event's amplitude will be the maximum magnification, $A_\text{max}$. Time of peak $t_0$, or time of closest approach, will be the time when magnification reaches $A_\text{max}$. Then, the time of planetary anomaly $t_\text{p}$ will be the time when the anomaly is occuring. The timescale of the anomaly, $t_\text{E,p}$, is half of the time of the duration of the planetary anomaly, while the Einstein timescale $t_\text{E}$ will be half of the duration of when the event's magnification is larger than $A=1.34$.

Now let's talk about the other event parameters. You should be familiar with the source radius scaled in Einstein radii, $\rho$, from the Chapter 3 lecture on the finite source effect. Below, we present an illustration of parameters connected to the binary lens model: planet-star separation $s$ and the angle of the source trajectory $\alpha$.

<center><img src="https://github.com/rges-pit/minicourses/blob/main/chapter4/images/Yee_sagan_planet_params.png?raw=true" width="500"/></center>
<center>Plot from Jennifer Yee's presentation during the Sagan Exoplanet Workshop 2017 for this exercise.</center>

The final parameter connected to the binary lens we will need to define is the mass ratio $q$, which is defined as:

$q = \frac{M_\text{planet}}{M_\text{source}}$

## <font face="Helvetica" size="6"> 4. Parameters of the stellar event </font>

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

First, we will determine the parameters of the stellar events, ignoring the planetary anomaly. Run the cell below and use the sliders to find the values of the single lens parameters.

In [ ]:
from ipywidgets import interact, FloatSlider

# Define the interactive plot function
def plot_with_lines(vline_pos, hline_pos):
    fig = plt.figure(figsize=(10, 6))
    plt.errorbar(df['HJD-2450000'], df['I_Mag'], yerr=df['Err'], marker='o', color='black', ls='', markersize=1)
    plt.gca().invert_yaxis()
    plt.xlabel('HJD-2450000')
    plt.ylabel('I-Mag')
    
    ## Vertical and horizontal lines controlled by sliders
    plt.axvline(x=vline_pos, color='red', linestyle='--', linewidth=2, label=f'vline={vline_pos:.2f}')
    plt.axhline(y=hline_pos, color='blue', linestyle='--', linewidth=2, label=f'hline={hline_pos:.2f}')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Create sliders with appropriate ranges based on the data
x_min, x_max = df['HJD-2450000'].min(), df['HJD-2450000'].max()
y_min, y_max = df['I_Mag'].min(), df['I_Mag'].max()  # Note: inverted because y-axis is inverted

# Create interactive plot with sliders
interact(plot_with_lines,
         vline_pos=FloatSlider(value=t0i, min=x_min, max=x_max, step=0.1, description='Vertical Line:', style={'description_width': 'initial'}),
         hline_pos=FloatSlider(value=df['I_Mag'].min(), min=y_min, max=y_max, step=0.01, description='Horizontal Line:', style={'description_width': 'initial'}));

We have to determine several parameters:
- t_0 is the time of the peak of the main event,
- mag_0 is the magnitude of the event at baseline,
- delta_mag_max is the amplitude between the baseline and the main event's peak,
- A_max is the amplification at the peak of the main event,
- u_0 is the impact parameter at the peak of the main event.

Use the plot above to find those values. You can help yourself with the vertical and horizontal lines, by editing their positions.

In [ ]:
# Time of the peak of the event
t_0 = 6141.38

# Baseline magnitude
mag_0 = 16.32

# Change in magnitude
delta_mag_max = (16.32 - 15.55)

# Maximum magnification
A_max = 10**(delta_mag_max/2.5)

# Impact parameter
u_0 = np.sqrt(2.0*(-1 + A_max/np.sqrt(A_max**2 - 1)))

print(delta_mag_max, A_max, u_0)

Now, we'd like to determine the Einstein timescale, t_E.

First, you have to find what the amplitude is when the amplification A is equal to 1.34. That is when the source is at the Einstein radius $\theta_E$.

Once you found the amplitude delta_mag at $A=1.34$, you can use the plot above to find the time when this amplitude is reached. Then, you can subtract that value from the time of peak and get the Einstein timescale, t_E.

In [ ]:
# Change in magnitude for A = 1.34
delta_mag = 2.5*np.log10(1.34)
# Time at delta_mag
t = 6189.98
# Einstein timescale
t_E = np.abs(t_0 - t)

print(t_E)

## <font face="Helvetica" size="6"> 5. Parameters of the planet </font>

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">  

Now we will focus on the planetary anomaly.

In [ ]:

# Create interactive plot with sliders
interact(plot_with_lines,
         vline_pos=FloatSlider(value=t0i, min=x_min, max=x_max, step=0.1, description='Vertical Line:', style={'description_width': 'initial'}),
         hline_pos=FloatSlider(value=df['I_Mag'].min(), min=y_min, max=y_max, step=0.01, description='Horizontal Line:', style={'description_width': 'initial'}));

### <font face="Helvetica" size="5"> 5.1 Where is the planet? </font>

First we have to find t_planet, the time when the planetary anomaly occurs. Use the plot above to determine that.

In [ ]:
# Time of the planet perturbation
t_planet = 6117.08

Now, based on the information gained above we can find the position of the images.

In [ ]:
# Time scaled to the Einsein timescale
tau = np.abs(t_planet - t_0) / t_E

# source-lens separation
u = np.sqrt(u_0**2 + tau**2)

# Position of the images
y_plus = 0.5 * np.sqrt(u**2 + 4.) + u
y_minus = 0.5 * np.sqrt(u**2 + 4.) - u

print(y_plus, y_minus)

### <font face="Helvetica" size="5"> 5.2 Find planet-star separation and the angle of the source trajectory </font>

To find the planet-star separation, we have to determine whether minor or major images are perturbed.

- Minor image perturbation occurs when there is a dip in the light curve during a planetary anomaly.

If the minor image is perturbed: $s = y_-$.

<center><img src="https://github.com/rges-pit/minicourses/blob/main//chapter4/images/Yee_sagan_minor_pert.png?raw=true" width="500"/></center>
<center>Minor image perturbation light curve, plot from Jennifer Yee's presentation during the Sagan Exoplanet Workshop 2017 for this exercise.</center>

- Major image perturbation occurs when there is a bump in the light curve during a planetary anomaly.

If the major image is perturbed: $s = y_+$

<center><img src="https://github.com/rges-pit/minicourses/blob/main/chapter4/images/Yee_sagan_major_pert.png?raw=true" width="500"/></center>
<center>Major image perturbation light curve, plot from Jennifer Yee's presentation during the Sagan Exoplanet Workshop 2017 for this exercise.</center>



⚠️ If your bump has a little dip that doesn't go back to the level of the main event, it would still be a major image perturbation rather than a minor image perturbation. The dip between two peaks can occur when the source is travelling inside the major image caustic.

In [ ]:
# planet-star separation
s = y_plus
print(s)

Now we can find the value of the angle of the source trajectory, alpha, using values we determined earlier. Due to different geometric conventions, the correct value might be $\frac{\pi}{2}$ or $\pi$ away from the value you calculated.

In [ ]:
# angle between trajectory and binary axis
alpha = np.arctan2(u_0, tau)

### <font face="Helvetica" size="5"> 5.3 Find  source radius and planet-star mass ratio </font>

Finding the planet-mass ratio and source radius depends on the event regime. First, you have to identify to which regime your event belongs.

If your event has a minor image perturbation, move to the [**Minor**](####-Minor-image,-rho-<-caustic) section.

If your event has a major image perturbation, you have to determine whether the source radius is larger or smaller than the caustic.
- If there are two distinct peaks, move to the [**Major, $\rho$< caustic**](####-Major-image,-rho-<-caustic) section.
- If there are no dips and the peaks are merged, move to the [**Major, $\rho$> caustic**](####-Major-image,-rho->-caustic) section.

<center><img src="https://github.com/rges-pit/minicourses/blob/main/chapter4/images/Yee_sagan_regimes.png?raw=true" width="500"/></center>
<center>Three regimes, this figure comes from Jennifer Yee's presentation for this exercise at the Sagan Summer Exoplanet Workshop 2017.</center>

#### <font face="Helvetica" size="4"> Minor image, $\rho$< caustic </font>

In this case, there is a dip in your light curve. $\rho$ is smaller than the caustic.
We will follow [**Han, 2006**](https://ui.adsabs.harvard.edu/abs/2006ApJ...638.1080H/abstract).
Look at Figure 2 from this paper. This is how the caustics look in your case.

<center><img src="https://github.com/rges-pit/minicourses/blob/main/chapter4/images/Han_2006_minor.png?raw=true" width="500"/></center>
<center>Minor image perturbation caustics, Han, 2006, Figure 2.</center>

The minor image caustic is shaped like two triangles.

You will have to now estimate the path of the source through the caustic(s). Use the alpha angle you determined earlier and the features in the light curve.
- How many caustic crossings are there? They manifest as sharp spikes in magnitude.
- Where is the dip? How long does it last?

First, we will find the source radius in Einstein radii.
We have to estimate the duration of the caustic crossing feature in the light curve.
If there's more than one, focus on one of them.

In [ ]:
# Create interactive plot with sliders
interact(plot_with_lines,
         vline_pos=FloatSlider(value=t0i, min=x_min, max=x_max, step=0.1, description='Vertical Line:', style={'description_width': 'initial'}),
         hline_pos=FloatSlider(value=df['I_Mag'].min(), min=y_min, max=y_max, step=0.01, description='Horizontal Line:', style={'description_width': 'initial'}));

In [ ]:
# length of one caustic feature
delta_t_caustic = np.abs(6119.28 - 6123.68) # Time it takes the source to cross the caustic
t_star = delta_t_caustic / 2.

# Now we can determine rho
rho = t_star / t_E

Now we will find the planet-star mass ratio, q. We will use the simplified equation (equation number 4 from the [**Handy equations**](###-Handy-equations) section).

Have a look at the length of the anomaly in the light curve. It occurs when the source is travelling between the two caustic triangles.

In [ ]:
# Create interactive plot with sliders
interact(plot_with_lines,
         vline_pos=FloatSlider(value=t0i, min=x_min, max=x_max, step=0.1, description='Vertical Line:', style={'description_width': 'initial'}),
         hline_pos=FloatSlider(value=df['I_Mag'].min(), min=y_min, max=y_max, step=0.01, description='Horizontal Line:', style={'description_width': 'initial'}));

In [ ]:
# Length of the anomaly
delta_tau = np.abs(6121.38 - 6112.58)
tE_planet = delta_tau / 2.

# Finally, mass ratio q is:
q = (tE_planet/t_E)**2 - rho**2

#### <font face="Helvetica" size="4"> Major image, $\rho$ < caustic </font>

In this case, there are distinct peaks in your light curve. $\rho$ is smaller than the caustic.
We will follow [**Han, 2006**](https://ui.adsabs.harvard.edu/abs/2006ApJ...638.1080H/abstract).
Look at Figure 1 from this paper. This is how the caustic looks in your case.

<center><img src="https://github.com/rges-pit/minicourses/blob/main/chapter4/images/Han_2006_major.png?raw=true" width="500"/></center>
<center>Major image perturbation caustics, Han, 2006, Figure 1.</center>

The major image caustic is shaped like a diamond.

You will have to now estimate the path of the source through the caustic. Use the alpha angle you determined earlier, as well as the features in the light curve.
- How many caustic crossings are there? They manifest as sharp spikes in magnitude.

First, we will find the source radius in Einstein radii.
We have to estimate the duration of the caustic crossing feature in the light curve.
If there's more than one, focus on one of them.

In [ ]:
# Create interactive plot with sliders
interact(plot_with_lines,
         vline_pos=FloatSlider(value=t0i, min=x_min, max=x_max, step=0.1, description='Vertical Line:', style={'description_width': 'initial'}),
         hline_pos=FloatSlider(value=df['I_Mag'].min(), min=y_min, max=y_max, step=0.01, description='Horizontal Line:', style={'description_width': 'initial'}));

In [ ]:
# Length of one caustic feature
# Time it takes the source to cross the caustic
delta_t_caustic =  

t_star = delta_t_caustic / 2.

# Now we can determine rho
rho = t_star / t_E

print(f"rho = {rho:.6f}")

Now we will find the planet-star mass ratio, q. We will use the simplified equation (equation number 4 from the [**Handy equations**](###-Handy-equations) section).

Have a look at the length of the anomaly in the light curve. It occurs when the source is travelling through the inside of the diamond.

In [ ]:
# Create interactive plot with sliders
interact(plot_with_lines,
         vline_pos=FloatSlider(value=t0i, min=x_min, max=x_max, step=0.1, description='Vertical Line:', style={'description_width': 'initial'}),
         hline_pos=FloatSlider(value=df['I_Mag'].min(), min=y_min, max=y_max, step=0.01, description='Horizontal Line:', style={'description_width': 'initial'}));

In [ ]:
# Length of the anomaly
delta_tau = 


# You will have to transform the equations provided a little bit. Put the calculations below.
# From equation 4: t_E,planet = t_E * sqrt(rho^2 + q^2)
# Solving for q: q = sqrt((t_E,planet/t_E)^2 - rho^2)

# Finally, mass ratio q is:

q = 
print(f"q = {q:.6e}")


#### <font face="Helvetica" size="5"> Major image, $\rho$ > caustic </font>

In this case, the peaks are merged in your light curve, and there are no dips. $\rho$ is larger than the caustic.
We will follow [**Gould & Gaucherel, 1997**](https://ui.adsabs.harvard.edu/abs/1997ApJ...477..580G/abstract).
Look at Figure 1 from this paper. This is how the caustic looks in your case.

<center><img src="https://github.com/rges-pit/minicourses/blob/main/chapter4/images/Gould_Gaucherel_1997.png?raw=true" width="500"/></center>
<center>Major image perturbation caustic, Gould & Gaucherel, 1997, Figure 1.</center>

The major image caustic is shaped like a diamond, but the source is large compared to it, and caustic entry and exit peaks merge.

You will have to now estimate the change in the magnitude for the caustic, delta_mag_c. This is the "height" of the bump in the light curve in magnitudes. You count the "height" from the main event light curve.

In [ ]:
# Change in the magnitude for the caustic
delta_mag_c =

Based on delta_mag_c you can now find the amplification during the planetary anomaly, A_p.

In [ ]:
# Insert the formula for magnification
A_p =

Finally, we can use this equation from [**Gould & Gaucherel, 1997**](https://ui.adsabs.harvard.edu/abs/1997ApJ...477..580G/abstract) to determine the mass ratio q:

$A_p = \frac{2q}{\rho^2}$

In [ ]:
# Insert formula for mass ratio q
q =

## <font face="Helvetica" size="6"> 6. Summary </font>

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

Let's display all the found parameters. We will show some of them in logarithmic form since this is a notation you can encounter in many papers.

Some of parameters, such as Einstein timescale $t_E$, source radius $\rho$, separation $s$ and mass ratio $q$ are easier to probe in log space, so have that in mind when you'll be tackling some of the events in the future.

In [ ]:
params = [t_0, u_0, t_E, rho, s, q, alpha]
params_labels = ['t_0', 'u_0', 't_E',
                 'rho', 's', 'q', 'alpha']
log_params = [t_0, u_0, np.log10(t_E), np.log10(rho), np.log10(s), np.log10(q), alpha]
log_params_labels = ['t_0', 'u_0', 'log_{10}(t_E)',
                     'log_{10}(rho)', 'log_{10}(s)', 'log_{10}(q)', 'alpha']


print('Your event parameters:')
for i in range(len(params)):
    print('%s = %.3f'%(params_labels[i], params[i]))

print('\nYour event parameters (log-space):')
for i in range(len(params)):
    print('%s = %.3f'%(log_params_labels[i], log_params[i]))

## <font face="Helvetica" size="6"> A. Handy equations </font>

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

### 1. Magnitude and amplification

$m_1 - m_2 = -2.5 log_{10}(\frac{f_1}{f_2}) \approx -2.5 log_{10}(A)$

### 2. Amplification and impact parameter

$A = \frac{u^2 + 2}{u\sqrt{u^2 + 4}} \approx \frac{1}{u}$

Approximation assumes $u << 1$.

### 3. Impact parameter at any given time

$u = \sqrt{(\frac{t-t_0}{t_E})^2 + u_0^2}$

### 4. Time of planetary anomaly and mass ratio

$t_\text{E, planet} = t_\text{E}\sqrt{\rho^2 + q^2}$,

$t_\text{E, planet} = \frac{1}{2}\Delta t_\text{planet}$

## About this Notebook

**Authors:** Katarzyna Kruszyńska, Arjun Murlidhar, Amber Malpas, Meet J. Vyas <br>
**Maintainers:** RGES-PIT Working Group 9  
**Last Updated:** 20 Nov 2025  
**Contact:** murlidhar.4@buckeyemail.osu.edu

## Citations

If you use this notebook for published research, please cite the
authors. Follow this links for more information about citing:

* [Citing **Roman Microlensing Data Challenge 2026 Notebooks**](https://github.com/rges-pit/data-challenge-notebooks/blob/main/zenodo.txt)

[Top of Page](#top)
<!-- Footer Start -->

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/roman_notebooks/refs/heads/main/stsci_logo2.png" alt="Space Telescope Logo" width="200px"/>

<!-- Footer End -->

## Section 5 Answers Summary

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

### Parameters Determined from Visual Inspection

#### 5.1 Planet Location
- **t_planet** (time of planetary anomaly): `6117.08`
- **tau** (time scaled to Einstein timescale): `np.abs(t_planet - t_0) / t_E`
- **u** (source-lens separation): `np.sqrt(u_0**2 + tau**2)`
- **y_plus** (major image position): `0.5 * np.sqrt(u**2 + 4.) + u`
- **y_minus** (minor image position): `0.5 * np.sqrt(u**2 + 4.) - u`

#### 5.2 Planet-Star Separation and Trajectory Angle
- **s** (planet-star separation): `y_plus`
- **alpha** (angle of source trajectory): `np.arctan2(u_0, tau)`

#### 5.3 Source Radius and Mass Ratio (Minor Image Case)
Based on the minor image perturbation analysis:

- **Caustic crossing times**: `6119.28` and `6123.68`
- **delta_t_caustic**: `np.abs(6119.28 - 6123.68)` = 4.4 days
- **t_star**: `delta_t_caustic / 2` = 2.2 days
- **rho** (source radius): `t_star / t_E`

- **Anomaly duration times**: `6121.38` and `6112.58`
- **delta_tau**: `np.abs(6121.38 - 6112.58)` = 8.8 days
- **tE_planet**: `delta_tau / 2` = 4.4 days
- **q** (mass ratio): `(tE_planet/t_E)**2 - rho**2`

These values were obtained from visual inspection of the light curve using the interactive sliders to measure the duration of caustic crossings and the planetary anomaly.